# File Ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [ ]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, behavior

Manual entry

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass(prompt='Database password:')
dj.conn()

### BPod Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [ ]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir

bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

### Initial check of tables

In [ ]:
from adamacs.pipeline import session, event, trial

session.Session.delete()


print('Sessions:', len(session.Session()))
print('Trials  :', len(trial.Trial()))
print('Events  :', len(event.Event()))

In [ ]:
event.EventType.delete()

## Automated BPod ingestion

The function is designed ask for a confirmation before entered into the schema.

In [ ]:
bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_object = Bpodfile(bpod_path)
bpod_object.ingest()

In [ ]:
subject.Subject()

Check that insertion worked:

In [ ]:
trial.TrialEvent & 'trial_id=0'

In [ ]:
trial.

In [ ]:
event.EventType()

We can also interact with bpod objects. For example:

In [ ]:
bpod_object.trial(1).attributes

In [ ]:
bpod_object.trial_data

# Add Harp recording

In [ ]:
from adamacs.pipeline import behavior, event
event_recording = event.BehaviorRecording.fetch('KEY')[0]
behavior.HarpRecording()

In [ ]:
behavior.HarpRecording.populate()

In [ ]:
behavior.HarpRecording.Channel()